In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

In [28]:
train_csv = "./data/train.csv"

In [29]:
train = pd.read_csv(train_csv)

In [30]:
train.head()

,Unnamed: 0,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [31]:
train.drop("Unnamed: 0", axis=1, inplace=True)

In [32]:
# going to drop the week start date column because it is redundant
train.drop("week_start_date", axis=1, inplace=True)

In [33]:
pd.set_option('display.max_columns', 25)

In [34]:
train = pd.get_dummies(train)

In [35]:
train.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,city_iq,city_sj
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,0,1
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,1
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,0,1
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,0,1
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,0,1


#### The gameplan:
Using NN to create an "index" from the pixel index. Polynomial all the weather features. lasso

But first, just one off the cuff NN

#### Neural Network

In [45]:
target = "total_cases"
columns = [x for x in train.columns if x != target]

In [48]:
neuralX = train[columns]
neuralY = train[target]

In [49]:
neuralX_train, neuralX_test, neuraly_train, neuraly_test = train_test_split(neuralX, neuralY)

In [50]:
neural_ss = StandardScaler()
neuralXs_train = neural_ss.fit_transform(neuralX_train)
neuralXs_test = neural_ss.transform(neuralX_test)

In [51]:
neuralXs_train.shape

(899, 24)

In [56]:
network = Sequential()

network.add(Dense(24, activation="relu", input_dim=neuralXs_train.shape[1], kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(576, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(24, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(1, activation=None, kernel_regularizer=regularizers.l1(.01)))

network.compile(loss="mean_absolute_error", optimizer="adam")

network.fit(neuralXs_train, neuraly_train, validation_data=(neuralXs_test, neuraly_test), epochs=500)


Train on 899 samples, validate on 300 samples
Epoch 1/500
899/899 [==============================] - 1s 733us/step - loss: 32.0151 - val_loss: 27.9156
Epoch 2/500
899/899 [==============================] - 0s 64us/step - loss: 25.5533 - val_loss: 22.6353
Epoch 3/500
899/899 [==============================] - 0s 76us/step - loss: 22.2267 - val_loss: 20.0049
Epoch 4/500
899/899 [==============================] - 0s 76us/step - loss: 20.0638 - val_loss: 18.8236
Epoch 5/500
899/899 [==============================] - 0s 75us/step - loss: 19.1633 - val_loss: 18.2682
Epoch 6/500
899/899 [==============================] - 0s 73us/step - loss: 18.1191 - val_loss: 17.8192
Epoch 7/500
899/899 [==============================] - 0s 72us/step - loss: 18.3385 - val_loss: 17.5235
Epoch 8/500
899/899 [==============================] - 0s 63us/step - loss: 17.6740 - val_loss: 17.3562
Epoch 9/500
899/899 [==============================] - 0s 70us/step - loss: 17.6841 - val_loss: 17.2342
Epoch 10/500
899/

899/899 [==============================] - 0s 65us/step - loss: 15.0041 - val_loss: 15.2367
Epoch 79/500
899/899 [==============================] - 0s 72us/step - loss: 14.9261 - val_loss: 15.1756
Epoch 80/500
899/899 [==============================] - 0s 70us/step - loss: 14.9222 - val_loss: 15.1590
Epoch 81/500
899/899 [==============================] - 0s 70us/step - loss: 15.4986 - val_loss: 15.1728
Epoch 82/500
899/899 [==============================] - 0s 71us/step - loss: 15.1510 - val_loss: 15.1481
Epoch 83/500
899/899 [==============================] - 0s 71us/step - loss: 15.4499 - val_loss: 15.1392
Epoch 84/500
899/899 [==============================] - 0s 81us/step - loss: 14.7653 - val_loss: 15.1536
Epoch 85/500
899/899 [==============================] - 0s 74us/step - loss: 15.1776 - val_loss: 15.1156
Epoch 86/500
899/899 [==============================] - 0s 67us/step - loss: 15.2668 - val_loss: 15.0961
Epoch 87/500
899/899 [==============================] - 0s 62us/step

899/899 [==============================] - 0s 52us/step - loss: 14.7419 - val_loss: 14.7990
Epoch 156/500
899/899 [==============================] - 0s 52us/step - loss: 14.8939 - val_loss: 14.7915
Epoch 157/500
899/899 [==============================] - 0s 51us/step - loss: 14.9363 - val_loss: 14.8051
Epoch 158/500
899/899 [==============================] - 0s 51us/step - loss: 14.5342 - val_loss: 14.8197
Epoch 159/500
899/899 [==============================] - 0s 51us/step - loss: 14.8198 - val_loss: 14.8369
Epoch 160/500
899/899 [==============================] - 0s 52us/step - loss: 15.1301 - val_loss: 14.7805
Epoch 161/500
899/899 [==============================] - 0s 51us/step - loss: 14.9503 - val_loss: 14.7718
Epoch 162/500
899/899 [==============================] - 0s 55us/step - loss: 14.9176 - val_loss: 14.7627
Epoch 163/500
899/899 [==============================] - 0s 52us/step - loss: 14.8681 - val_loss: 14.7831
Epoch 164/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 51us/step - loss: 14.6217 - val_loss: 14.6431
Epoch 233/500
899/899 [==============================] - 0s 54us/step - loss: 14.9076 - val_loss: 14.6418
Epoch 234/500
899/899 [==============================] - 0s 53us/step - loss: 14.5954 - val_loss: 14.6094
Epoch 235/500
899/899 [==============================] - 0s 52us/step - loss: 14.5970 - val_loss: 14.6092
Epoch 236/500
899/899 [==============================] - 0s 51us/step - loss: 14.8721 - val_loss: 14.6090
Epoch 237/500
899/899 [==============================] - 0s 52us/step - loss: 14.4887 - val_loss: 14.6476
Epoch 238/500
899/899 [==============================] - 0s 51us/step - loss: 14.8820 - val_loss: 14.6820
Epoch 239/500
899/899 [==============================] - 0s 51us/step - loss: 14.5749 - val_loss: 14.6534
Epoch 240/500
899/899 [==============================] - 0s 51us/step - loss: 14.4816 - val_loss: 14.6659
Epoch 241/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 73us/step - loss: 14.6761 - val_loss: 14.4906
Epoch 310/500
899/899 [==============================] - 0s 85us/step - loss: 14.4818 - val_loss: 14.4906
Epoch 311/500
899/899 [==============================] - 0s 73us/step - loss: 14.5109 - val_loss: 14.4772
Epoch 312/500
899/899 [==============================] - 0s 65us/step - loss: 14.7000 - val_loss: 14.4904
Epoch 313/500
899/899 [==============================] - 0s 65us/step - loss: 14.4726 - val_loss: 14.4792
Epoch 314/500
899/899 [==============================] - 0s 78us/step - loss: 14.8026 - val_loss: 14.4825
Epoch 315/500
899/899 [==============================] - 0s 78us/step - loss: 14.6782 - val_loss: 14.4859
Epoch 316/500
899/899 [==============================] - 0s 80us/step - loss: 14.3801 - val_loss: 14.4745
Epoch 317/500
899/899 [==============================] - 0s 65us/step - loss: 14.4945 - val_loss: 14.5152
Epoch 318/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 82us/step - loss: 14.4009 - val_loss: 14.4487
Epoch 387/500
899/899 [==============================] - 0s 82us/step - loss: 14.6393 - val_loss: 14.4610
Epoch 388/500
899/899 [==============================] - 0s 72us/step - loss: 14.5868 - val_loss: 14.4519
Epoch 389/500
899/899 [==============================] - 0s 55us/step - loss: 14.1560 - val_loss: 14.4648
Epoch 390/500
899/899 [==============================] - 0s 62us/step - loss: 14.4993 - val_loss: 14.4514
Epoch 391/500
899/899 [==============================] - 0s 62us/step - loss: 14.6803 - val_loss: 14.4322
Epoch 392/500
899/899 [==============================] - 0s 61us/step - loss: 14.4481 - val_loss: 14.4148
Epoch 393/500
899/899 [==============================] - 0s 59us/step - loss: 14.7383 - val_loss: 14.3903
Epoch 394/500
899/899 [==============================] - 0s 74us/step - loss: 14.4605 - val_loss: 14.3691
Epoch 395/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 65us/step - loss: 14.6496 - val_loss: 14.4365
Epoch 464/500
899/899 [==============================] - 0s 65us/step - loss: 14.5984 - val_loss: 14.3850
Epoch 465/500
899/899 [==============================] - 0s 75us/step - loss: 14.4379 - val_loss: 14.3751
Epoch 466/500
899/899 [==============================] - 0s 70us/step - loss: 14.7369 - val_loss: 14.3784
Epoch 467/500
899/899 [==============================] - 0s 66us/step - loss: 14.5538 - val_loss: 14.3941
Epoch 468/500
899/899 [==============================] - 0s 63us/step - loss: 14.7196 - val_loss: 14.3823
Epoch 469/500
899/899 [==============================] - 0s 79us/step - loss: 14.1634 - val_loss: 14.3411
Epoch 470/500
899/899 [==============================] - 0s 72us/step - loss: 14.6828 - val_loss: 14.3675
Epoch 471/500
899/899 [==============================] - 0s 72us/step - loss: 14.2024 - val_loss: 14.4185
Epoch 472/500
899/899 [==============================] - 0s 

In [58]:
# neural networks are like cheating
# the high score on drivendata is 13 MAE